In [1]:
import utils
import pandas as pd
import random
from datetime import datetime
import math
from collections import Counter
from functools import reduce

big_group_name_list = []
big_group_index_list = []
stock_data = pd.read_excel('원자재내역.XLS')
stock_data = stock_data.sort_values(by=['ALLOY','TEMPER','포장중량','실폭','생산일자'],ascending=False).reset_index()

origin_data = pd.read_excel('폭조합용 세부자료_V2_190916.xls',sheet_name = '입력시트')
origin_data=origin_data.fillna(0)
origin_data = origin_data.drop(['팀'],axis=1)

## 데이터 정리 -> 폭조합 x, 제품코드 0, 생산량 0 행 제거
drop_index=[]
for i in range(len(origin_data)):
    if origin_data['제품코드'][i]==0 or origin_data['폭조합'][i]=='X' or origin_data['생산량'][i] == 0:
        drop_index.append(i)
origin_data = origin_data.drop(drop_index)
origin_data = origin_data.reset_index(drop=True)

## 횟수 설정
origin_data['횟수']=0
for i in range(len(origin_data)):
    thickness = origin_data['두께'][i]
    width = origin_data['폭'][i]
    length = origin_data['길이(기준)'][i]
    goal_weight = origin_data['생산량'][i]
    repeat = utils.calculateRepeat(thickness,width,length,goal_weight)
    if utils.check_doubling(origin_data['권취'][i]) and (repeat)%2 !=0: ## 더블링인 경우 2의 배수로 저장
        repeat+=1
    origin_data['횟수'][i] = repeat
origin_data['const_횟수'] = origin_data['횟수']
origin_data['addition_횟수'] = 0 
for i in range(len(origin_data)):
    if origin_data['횟수'][i] % 2 ==0:
        origin_data['addition_횟수'][i] = (origin_data['횟수'][i]*0.5)+(origin_data['횟수'][i]*0.5)%2
    else:
        origin_data['addition_횟수'][i] = (origin_data['횟수'][i]*0.5)
origin_data['const_생산량'] = origin_data['생산량']
origin_data['const_addition_횟수'] = origin_data['addition_횟수']
origin_data['addition_횟수'] = 0
origin_data['addition_생산량'] = origin_data['생산량']*0.5
origin_data['const_addition_생산량'] = origin_data['addition_생산량']
origin_data = origin_data.sort_values(by=['ALLOY_1','권취','TEMPER','원자재_M','원자재_T','두께','길이(기준)','내경','코아','폭'],\
                                      ascending=False).reset_index(drop=True)

sorted_data = origin_data
#재고기반 grouping
big_group_name_list=[]
big_group_index_list=[]
for i in range(sorted_data.shape[0]):
    #temp=""
    temp=str(sorted_data['ALLOY_1'][i])
    temp=utils.translate_alloy(temp)
    #temp+=str(sorted_data['제품폭'][i])
    check = 0
    for j in range(len(big_group_name_list)):
        if(temp==big_group_name_list[j]):
            check +=1
            break
    if(check==0):
        big_group_name_list.append(temp)
        big_group_index_list.append(i)
print(big_group_name_list)
print(big_group_index_list)


/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


['BRW04', 'A8021', 'A8079', 'F308', 'F309', 'A3003', 'A1100', 'A1235', 'A1050']
[0, 1, 9, 146, 151, 153, 157, 200, 212]


In [16]:
#재고 data index1
stock_name_list=[]
stock_index_list=[]

##group 선택
group_name =  'A1235'#'A8021' # big_group_name_list[-3] #  

for i in range(stock_data.shape[0]):
    #temp=""
    temp=str(stock_data['ALLOY'][i])
    #temp+=str(sorted_data['제품폭'][i])
    check = 0
    for j in range(len(stock_name_list)):
        if(temp==stock_name_list[j]):
            check +=1
            break
    if(check==0):
        stock_name_list.append(temp)
        stock_index_list.append(i)
        
print(stock_name_list)
print(stock_index_list)

#주문 data에서 원자재에 맞게 추출
for i in range(len(big_group_name_list)):
    if group_name == (big_group_name_list[i]):
        stock_index = i
        break
if i != len(big_group_name_list)-1:
    stock_group_df = sorted_data[big_group_index_list[stock_index]:big_group_index_list[stock_index+1]].reset_index(drop=True)       
else:
    stock_group_df = sorted_data[big_group_index_list[stock_index]:len(sorted_data)].reset_index(drop=True)       

#stock_group_df = sorted_data
for i in range(len(stock_name_list)):
    if group_name == stock_name_list[i]:
        stock_index = i
        break
if i != len(stock_name_list)-1:
    stock_df = stock_data[stock_index_list[stock_index]:stock_index_list[stock_index+1]].reset_index(drop=True)
else:
    stock_df = stock_data[stock_index_list[stock_index]:len(stock_data)].reset_index(drop=True)
    

    
stock_group_df = stock_group_df.sort_values(by=['권취','TEMPER','원자재_M','원자재_T','두께','길이(기준)','내경','코아','폭'],\
                                            ascending=False).reset_index(drop=True) 


now = datetime.now()
time = str(now.year)+str(now.month)+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)+'_'
file_name=time+group_name+"_temp.xlsx"
#writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
#stock_group_df.to_excel(writer)
#writer.close()

##특성값으로 grouping
small_group_name =[]
small_group_index = []
for i in range(stock_group_df.shape[0]):
    temp= ""
    #temp+=str(stock_group_df['ALLOY'][i])+"-"
    temp+=stock_group_df['권취'][i]+"-"
    temp+=str(stock_group_df['TEMPER'][i])+"-" 
    temp+=str(stock_group_df['원자재_M'][i])+"-"
    temp+=str(stock_group_df['원자재_T'][i])+"-"
    temp+=str(stock_group_df['두께'][i])+"-"
    temp+=str(stock_group_df['내경'][i])+"-"
    temp+=str(stock_group_df['길이(기준)'][i])+"-"
    temp+=stock_group_df['코아'][i]
    check = 0
    for j in range(len(small_group_name)):
        if(temp==small_group_name[j]):
            check +=1
            break
    if(check==0):
        small_group_name.append(temp)
        small_group_index.append(i)

small_group_name.append("end")
small_group_index.append(len(stock_group_df))

## 원자재 결정        
big_group_name = []
big_group_index=[]
for i in range(stock_group_df.shape[0]):
    temp=str(stock_group_df['권취'][i])+'-'
    temp+=str(stock_group_df['TEMPER'][i])+"-"
    temp+=str(stock_group_df['원자재_M'][i])+"-"
    temp+=str(stock_group_df['원자재_T'][i])+"-"
    temp+=str(stock_group_df['두께'][i])
    check = 0
    for j in range(len(big_group_name)):
        if(temp==big_group_name[j]):
            check +=1
            break
    if(check==0):
        big_group_name.append(temp)
        big_group_index.append(i)

big_group_name.append("end")
big_group_index.append(len(stock_group_df))


possible_weight_in_group = []
#weight_list
extra_weight_list =[]
index = 0
i=0
while(i < len(big_group_index)-2):
    temp = []
    extra_temp = []
    while(big_group_index[i] <= small_group_index[index] and small_group_index[index] < big_group_index[i+1]):
        temp_sum = 0 
        extra_sum = 0
        for j in range(small_group_index[index],small_group_index[index+1]):
            temp_sum += stock_group_df['생산량'][j]
            #extra_sum += weight_list[j]
        index +=1
        temp.append(temp_sum)
        #extra_temp.append(extra_sum)
    if len(temp)!=0:
        possible_weight_in_group.append(temp)
        #extra_weight_list.append(extra_temp)
    else:
        i+=1
        
temp = []
extra_temp = []
while(big_group_index[i] <= small_group_index[index] and small_group_index[index] < len(stock_group_df)):
    temp_sum = 0
    extra_sum = 0
    if index < len(small_group_index)-1:
        for j in range(small_group_index[index],small_group_index[index+1]):
            temp_sum += stock_group_df['생산량'][j]
            #extra_sum += weight_list[j]
        index +=1
    else:
        for j in range(small_group_index[index],len(small_group_index)):
            temp_sum += stock_group_df['생산량'][j]
            #extra_sum += weight_list[j]
        break;
    temp.append(temp_sum)
    ##extra_temp.append(extra_sum)
    
if len(temp)!=0:
    possible_weight_in_group.append(temp)
    #extra_weight_list.append(extra_temp)
        
possible_weight = []
for i in range(len(big_group_index)):
    temp_sum =0
    if i < len(big_group_index)-1:
        for index in range(big_group_index[i],big_group_index[i+1]):
            temp_sum +=stock_group_df['생산량'][index]
    else:
        for index in range(big_group_index[i],len(stock_group_df)):
            temp_sum +=stock_group_df['생산량'][index]
    possible_weight.append(temp_sum)
        

impossible_weight = 0
total_weight = 0
count =-1
possible_list_name = []
for i in range(len(possible_weight_in_group)):
    for j in range(len(possible_weight_in_group[i])):
        count +=1
        if possible_weight_in_group[i][j] < 5:
            impossible_weight += possible_weight_in_group[i][j]
        else:
            possible_list_name.append(small_group_name[count])
print(impossible_weight)
for i in possible_weight:
    total_weight+= i
print(impossible_weight/total_weight)

total_stock_weight = 0
for i in range(len(stock_df)):
    total_stock_weight +=stock_df["포장중량"][i]/1000


['LPf21', 'F309', 'F308', 'BRW04', 'A8079', 'A8021', 'A3003', 'A1235', 'A1100', 'A1050']
[0, 2, 3, 7, 9, 183, 251, 263, 318, 471]
5.5
0.034482758620689655


In [42]:
CONST_OUT_OF_COUNT_NUM = 100000
RESIDUAL_RATE = 0.3

SUM_OF_SCOURE = 0 
SUM_OF_SCOURE_COUNT = 0
SUM_OF_TOTAL_COUNT = 0
stock_group_df['횟수'] = stock_group_df['const_횟수'] 
stock_group_df['addition_횟수'] = 0 #stock_group_df['const_addition_횟수'] 

stock_group_df['생산량'] = stock_group_df['const_생산량'] 
stock_group_df['addition_생산량'] = stock_group_df['const_addition_생산량'] 
#랜덤으로 값 추출
#조합 가능한 폭 list
selected_list = []
selected_stock=[]
selected_df_list=[]
total_order_num = len(stock_group_df)

count_zero=-1
sum_residual=0
sum_addition_weight = 0
selected_weight=0
total_weight= utils.sum_list(possible_weight)
total_count = utils.sum_list(stock_group_df['횟수'])
total_addition_count = 0 
current_count = 0 

best_selected_list = []
select_list = []
best_sum_weight = 0

next = False
reset_count = -1
while(reset_count<0): 
    reset_count +=1
    for r in range(len(big_group_index)-len(big_group_index),len(big_group_index)-1):#len(big_group_index)-1            
        start_index=0
        end_index = 0
              
        for i in range(len(small_group_index)): #'권취','TEMPER','두께'가 동일한 범위 선택
            if big_group_index[r] == small_group_index[i]: 
                start_index = i
            elif big_group_index[r+1] == small_group_index[i]:
                end_index = i
                break;
                
        #usued_stock_list = []
        
        ##남은 big group 내 남은 무게 계산
        big_group_total_weight = 0
        big_group_total_count = 0
        for i in range(small_group_index[start_index],small_group_index[end_index]):
            big_group_total_weight += stock_group_df['생산량'][i]
            if stock_group_df['횟수'][i]!= CONST_OUT_OF_COUNT_NUM:
                big_group_total_count += stock_group_df['횟수'][i]
        combi_try_count = -1
        standard_score = 100
        
        n=-1
        total_best_score = -CONST_OUT_OF_COUNT_NUM
        total_best_select_list = []
        total_best_stock_num = -1
        total_best_residual = CONST_OUT_OF_COUNT_NUM
        total_best_addition_weight = 0
        total_best_combi_weight = 0
        total_best_material_realweight = 0
        total_tri_count = 0
        need_new_material_soure = 0
        one_more = 0
     
        print("small_group_index[start_index],small_group_index[end_index]"\
                      ,small_group_index[start_index],small_group_index[end_index])
        print("big_group_total_count",big_group_total_count)
        
        range_weight = small_group_index[end_index] - small_group_index[start_index]
        
        while big_group_total_count > 0: #  and total_tri_count < len(stock_df)  
            combi_try_count +=1
            search_count = -1
            new_material = False
            
            ## 원자재 검색
            #n = random.randrange(0,len(stock_df))
            possible = False
            while 1:
                
                #n = utils.find_possible_index(selected_stock, len(stock_df),n)
                n += 1
                #n = random.randrange(0,len(stock_df))
                n = n%len(stock_df)
                ## 원자재 정보 setting
                material_code = stock_df['제품코드'][n]
                material_company=material_code[len(material_code)-1]
                material_temper = stock_df['TEMPER'][n]
                const_width = stock_df['실폭'][n]
                material_alloy = utils.translate_alloy(stock_df['ALLOY'][n]) 
                width = stock_df['실폭'][n]
                weight = float(stock_df['포장중량'][n])/1000
                thickness = stock_group_df['두께'][small_group_index[start_index]]
                cal_realweight= utils.calculateRealweight(thickness,weight)
                #if stock_group_df['원자재_M'][big_group_index[r]] == '무관':
                #    order_material = 'N'
                #else:
                order_material = stock_group_df['원자재_M'][big_group_index[r]]
                possible = utils.check_material(material_company,material_temper,thickness,\
                                          order_material,\
                                          stock_group_df['원자재_T'][big_group_index[r]],\
                                          stock_group_df['두께'][big_group_index[r]])
                if possible:
                    break
                
                
                search_count +=1
                if search_count > len(stock_df)+1:
                    ## 사용 가능한 원자재가 없을 경우 -> 새로운 조건을 넣어야함
                    new_material = True
                    possible = False
                    break
            
            #새로운 원자재를 구매해야하는 경우
                
            if need_new_material_soure >len(stock_df)*5:
                print("need new material!!!!%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
                break;
              
            if not possible and new_material:
                #print("not possible and new_material")
                need_new_material_soure +=1      
                
            else:     
                
                #print("material_code: ",material_code, n)
                combi_list=[]
                combi_index_list=[]
                combi_count_list=[]
                #print("width",width)
                SUM_OF_TOTAL_COUNT +=1
                for i in range(start_index,end_index):  #'권취','TEMPER','두께' 일치
                    
                    max_index_count = 0
                    
                    ## group_index_start, group_index_end 설정
                    group_index_start = small_group_index[i]
                    if i < len(small_group_index)-1:
                        group_index_end = small_group_index[i+1]
                    else:
                        group_index_end = small_group_index[i]
                        
                    ## 산업재, 박박, 후박, FIN 등
                    detail_code = stock_group_df['세부품목'][group_index_start]
                    ## 횟수 확인
                    count_index = []
                    max_index_count = 0
                    sum_count = 0 
                    for j in range(group_index_start,group_index_end):
                        
                        rand_choice = random.randrange(0,2)
                        ## 횟수가 없는 오더도 포함
                        if stock_group_df['횟수'][j]<CONST_OUT_OF_COUNT_NUM:
                            prob = stock_group_df['횟수'][j]#*stock_group_df['횟수'][j]
                            count_index.append((prob,j)) 
                            max_index_count += stock_group_df['횟수'][j]##횟수에 맞게 max_index_count 설정
                            sum_count += prob
                        
                        elif (stock_group_df['addition_횟수'][j] == 0 and\
                            stock_group_df['const_횟수'][j] > 10) :
                            count_index.append((1,j)) 
                            max_index_count += 1 
                            sum_count += 1
                             
                    temp_list=[]
                    temp_index_list=[]
                    temp_count_list=[]
                    ## 횟수가 양수일 경우 -> 최소폭 구함
                    if sum_count >0:
                        
                        count_index.sort(reverse=True)   
                        index_count = -1
                        index = -1
                        
                        min_width = stock_group_df['폭'][group_index_end-1]
                        initial_mim_width = utils.get_mim_width(1,thickness,material_alloy,detail_code)
                        max_index_count *=2
                        while (index_count < max_index_count and width >= min_width+initial_mim_width): 
                            #'ALLOY','권취','TEMPER','두께','길이','내경','코아','폭' 일치
                            temp_list.append([])
                            temp_index_list.append([])
                            temp_count_list.append([])
                            index_count += 1
                            index += 1
                            extra_width = width
                            
                            try_combi_count = 0
                            check_only_addition = True
                            num_of_combi = 0
                            pre_mim_width = 0
                            
                            
                            #addition_mim_width = initial_mim_width
                            while(extra_width >= min_width and try_combi_count<100):
                                ##횟수에 비례해서 선택
                                rand = random.randrange(0,sum_count)
                                for k in range(len(count_index)):
                                    rand -= count_index[k][0]
                                    if rand < 0:
                                        j = count_index[k][1]
                                        break
                                #j =0 
                                mim_width = utils.get_mim_width(num_of_combi+1,thickness,material_alloy,detail_code) 
                                addition_mim_width = mim_width - pre_mim_width
                                
                                if utils.check_push(temp_list[index],stock_group_df['폭'][j]+addition_mim_width,width):
                                    
                                    if stock_group_df['횟수'][j] < CONST_OUT_OF_COUNT_NUM:
                                        check_only_addition = False ## 추가 오더만으로 생산은 필요 없음
                                    temp_list[index].append(stock_group_df['폭'][j]+addition_mim_width)
                                    temp_count_list[index].append(stock_group_df['횟수'][j])
                                    temp_index_list[index].append(j) #해당 index 저장
                                    extra_width -= (stock_group_df['폭'][j]+addition_mim_width)
                                    num_of_combi +=1
                                    pre_mim_width = mim_width
                                try_combi_count+=1
                            ### special_addition_count_list -> 남은 생산량 있는데 입력 조합에 초과해서 들어갈 경우
                            ## special_addition_count_list[i] -> [초과횟수,index]
                            special_addition_count_list = [] 
                            
                            if len(temp_count_list[index]) > 0 and (not check_only_addition):
                                ## 중복 index가 가능한 조합인지 확인
                                count_index_list = Counter(temp_index_list[index])
                                for k in range(len(temp_index_list[index])):
                                    ## 중복되는 만큼 가능 횟수 나누어줌
                                    temp_count_list[index][k] = temp_count_list[index][k] // count_index_list[temp_index_list[index][k]]
                                    
                                    if temp_count_list[index][k] == 0: ## 초과 입력되는 경우
                                        temp_count_list[index][k] =2 ##doubling을 고려하여 2번
                                        find_index = False
                                        for a in range(len(special_addition_count_list)):
                                            if special_addition_count_list[a][1] == temp_index_list[index][k]: ##해당 index가 있을 경우
                                                special_addition_count_list[a][0] += 2 ##초과 횟수 추가
                                                find_index = True
                                                break;
                                                
                                        if not find_index: ##해당 index가 없을 경우 새롭게 생성 [초과횟수(2),index]
                                            special_addition_count_list.append([2,temp_index_list[index][k]])
                            
                            #본 조합이 좋은 조합이면 살리고 아니면 죽임
                            if not(utils.checkCombination(temp_list,temp_index_list,index,extra_width,width)) \
                                    or check_only_addition:
                                del temp_list[index]
                                del temp_index_list[index]
                                del temp_count_list[index]
                                index += -1
                                
                        #좋은 조합 모음 and 새로운 조합
                        if len(temp_list)!=0 and stock_group_df['길이(기준)'][j] >0:
                            temp_list.append(stock_group_df['길이(기준)'][j])#각 조합의 길이는 마지막에 넣음
                            combi_list.append(temp_list)
                            combi_index_list.append(temp_index_list)
                            combi_count_list.append(temp_count_list)
                

                ########조합 종목 선택
       
                #값 변환
                sorted_realweight_list = [] #무게, 횟수, index_list, count_list, combi_list, 가로 손실 길이 ,낭비량
               
                max_realweight = 0
                if len(combi_list)>0 :
                    for i in range(len(combi_list)):
                        #각 조합의 길이는 마지막에 있음
                        temp_length = combi_list[i][len(combi_list[i])-1]
                        for j in range(len(combi_list[i])-1): 
                            
                            min_repeat_count = min(combi_count_list[i][j])
                            #print(combi_index_list)
                            temp_w = utils.sum_list(combi_list[i][j])
                            temp = []
                            doubing = utils.check_doubling(stock_group_df['권취'][combi_index_list[i][0][0]])

                            for k in range(min_repeat_count):
                                if (doubing and (k+1)%2 ==0) or (not doubing): ##더블링 조건에 맞게 무게 계산
                                    ## 낭비량 계산 
                                    temp_residual = round(utils.realWeight(thickness,(width)-temp_w,temp_length,1,k+1)/1000,2)
                                    zero_list = []
                                    for l in range(len(combi_count_list[i][j])):
                                        zero_list.append(0)
                                        
                                    #무게, 횟수, index_list, count_list, combi_list, 추가 count_list ,가로 손실 길이 ,낭비량
                                    temp.append([round(utils.realWeight(thickness,temp_w,temp_length,1,k+1)/1000,2),k+1\
                                                 ,combi_index_list[i][j],combi_count_list[i][j],combi_list[i][j],zero_list,\
                                                 (width)-temp_w,temp_residual])
                                    
                            if len(temp)!=0:
                                temp.sort(reverse=True)
                                sorted_realweight_list.append(temp)
  
                #최적화, 제거 목록 선택
                if len(sorted_realweight_list)==0:
                    #standard_score -= 0.1
                    combi_try_count -=1
                    
                else:   
                    #원자재 무게보다 많이 나가는 조합 버림       
                    for i in range(len(sorted_realweight_list)):
                        del_index=0
                        for j in range(len(sorted_realweight_list[i])):
                            if sorted_realweight_list[i][j][0] > cal_realweight:
                                del_index = j
                            else:
                                sorted_realweight_list[i] = sorted_realweight_list[i][del_index:len(sorted_realweight_list[i])]
                                break;
                                
                    ## 초기 잔여량은 원자재 사용 무게 / 무게 순으로 sort
                    select_residual=CONST_OUT_OF_COUNT_NUM
                    #select_addition_weight=0
                    select_list=[]
                    sorted_realweight_list.sort(reverse=True)
                    start_combi=True
                    best_score = -CONST_OUT_OF_COUNT_NUM
                    combi_weight = 0
                    max_count = 1
                    
                    ##max_count 설정
                    for i in range(len(sorted_realweight_list)):
                        max_count += len(sorted_realweight_list[i])#*len(sorted_realweight_list[i])
                   
                    count = -1
                    max_count *=2
                    while(count<max_count):
                        count+=1
                        temp_list = []
                        combi_count = 0 
                        temp_total_extra = cal_realweight
                        temp_sum_redisual = 0
                        temp_try_count = -1
                        temp_combi_weight = 0
                        temp_extra_width = 0 
                           
                        while(combi_count<2 and temp_try_count<10):
                            
                            temp_try_count += 1
                            i = random.randrange(0,len(sorted_realweight_list))
                            j = random.randrange(0,len(sorted_realweight_list[i]))
                                
                            ## overlap check
                            check_overlap_index = False
                            temp_select_index = sorted_realweight_list[i][j][2][:]
                            temp_select_index.sort()
                            for a in range(len(temp_list)):
                                for b in range(len(temp_list[a])):
                                    temp_check_index_list = temp_list[a][2][:]
                                    temp_check_index_list.sort()
                                    if temp_check_index_list == temp_select_index:
                                        check_overlap_index = True
                                        break;
                                        
                                #추가로 더 넣을 수 있는지 확인
                            if temp_total_extra -(sorted_realweight_list[i][j][0]+sorted_realweight_list[i][j][-1]) >= -cal_realweight*0.02\
                                and (not check_overlap_index):
                            
                                temp_list.append(sorted_realweight_list[i][j])
                                temp_total_extra -= (sorted_realweight_list[i][j][0]+sorted_realweight_list[i][j][-1])
                                temp_sum_redisual += sorted_realweight_list[i][j][-1]
                                temp_combi_weight += sorted_realweight_list[i][j][0]
                                temp_extra_width += sorted_realweight_list[i][j][-2]
                                combi_count +=1
                                
                        temp_score = -CONST_OUT_OF_COUNT_NUM
                        temp_addition_weight = 0
                        
                        ## 추가 생산량이 있더라도 한번도 생산하지 않은 것들을 생산
                        used = True
                        for i in range(len(temp_list)):
                            for j in range(len(temp_list[i][2])):
                                k = temp_list[i][2][j]
                                if stock_group_df['횟수'][k] == stock_group_df['const_횟수'][k] or \
                                    (stock_group_df['횟수'][j] >5 and stock_group_df['횟수'][j] <9):#:#
                                    used = False
                                    break;
                                #elif (stock_group_df['횟수'][j] <3):
                                #    used = True
                                #    break;
                                        
                        if temp_total_extra <= cal_realweight*RESIDUAL_RATE or not(used): # len(temp_list)>0 : #and temp_total_extra >= cal_realweight*0.1
                            
                            pre_count = 0
                            addition_count = 0
                            addition_residual = 0
                            addition_weight = 0
                    
                            #무게, 횟수, index_list, count_list, combi_list, 추가 count_list, 가로 손실 길이 ,낭비량
                            ## 가장 빈도수가 낮은 조합 횟수 최대로 늘림
                            min_index= -1
                            min_count = CONST_OUT_OF_COUNT_NUM
                            for i in range(len(temp_list)):
                                if min_count >= temp_list[i][1]:
                                    min_index = i; min_count = temp_list[i][1];
                                
                                ## doubling체크
                            doubing = utils.check_doubling(stock_group_df['권취'][temp_list[min_index][2][0]])
                            temp_length = stock_group_df['길이(기준)'][temp_list[min_index][2][0]]
                            temp_w = utils.sum_list(temp_list[min_index][4])
                                
                                ## 계획을 초과하는 생산하는 횟수 따로 기록
                            while 1:
                                if ((doubing and (addition_count)%2 ==0) or (not doubing)): ##더블링 조건에 맞게 무게 계산
                                    addition_weight = round(utils.realWeight(thickness,temp_w,temp_length,1,addition_count)/1000,2)
                                    addition_residual = round(utils.realWeight(thickness,temp_list[min_index][-2],temp_length,1,addition_count)/1000,2)

                                    if temp_total_extra - addition_weight <= 0:
                                        if temp_total_extra -(addition_weight+addition_residual) < -cal_realweight*0.01:
                                            addition_count = pre_count
                                            addition_residual = round(utils.realWeight(thickness,temp_list[min_index][-2],temp_length,1,pre_count)/1000,2)
                                            addition_weight = round(utils.realWeight(thickness,temp_w,temp_length,1,pre_count)/1000,2)
                                        break;
                                    else:
                                        pre_count = addition_count

                                addition_count+=1

                            temp_list[min_index][1] += addition_count
                            temp_list[min_index][0] += addition_weight
                                       
                                ### ex) [0, 0, 0, 1] -> 나중에 그 횟수를 따로 처리
                            for i in range(len(temp_list[min_index][3])):
                                if stock_group_df['횟수'][temp_list[min_index][2][i]] != CONST_OUT_OF_COUNT_NUM: 
                                    temp_list[min_index][5][i]= min(temp_list[min_index][3][i]-temp_list[min_index][1],0)
                                else:
                                    temp_list[min_index][5][i]= -temp_list[min_index][1]

                                

                            ### 점수 계산에 필요한 값들 update            
                            temp_total_residual =(temp_total_extra-addition_weight+addition_residual)
                            temp_combi_weight += addition_weight
                            
                            #temp_addition_weight +=addition_weight
                            
                            addition_count = 0
                            combi_total_count = 0      
                            test=0
                            for i in range(len(temp_list)):
                                test += len(list(set(temp_list[i][2])))
                                combi_total_count += temp_list[i][1]*len(temp_list[i][2])
                                addition_count += -utils.sum_list(temp_list[i][5])
                                
                            ## special_addition_count_list 계산
                            if len(special_addition_count_list)!=0:
                                for i in range(len(special_addition_count_list)):
                                    addition_count +=\
                                    (special_addition_count_list[i][0]-stock_group_df['횟수'][special_addition_count_list[i][1]])
                            
                            
                            temp_score = 100 - (100*((temp_total_residual/cal_realweight)+2*(temp_extra_width/70))\
                                                +temp_extra_width*((combi_count-1)/3)+500*(addition_count/combi_total_count)) #100*(addition_weight/cal_realweight)
                            
                            temp_score += 5*((temp_combi_weight/cal_realweight)+((70-temp_extra_width)/70))\
                                            *(1/(addition_count*addition_count+1))
                                    
                        #새로운 조합이 더 좋을 경우
                        # 점수 비교
                        if best_score < temp_score:
    
                            ##중복 index 확인
                            no_overlap = True
                            temp_index_list = []

                            for i in range(len(temp_list)):
                                temp_index_list +=list(set(temp_list[i][2])) #사용되는 index list(중복 제거)

                            if temp_index_list != list(set(temp_index_list)): #다른 조합간에 겹치는 index 존재할 경우   
                                temp_index_list = list(set(temp_index_list)) #중복 제거 list 
                                test_dict = {}
                                for i in temp_index_list: #dict 초기화
                                    test_dict[i] = 0
                                    
                                for i in range(len(temp_list)):
                                    for j in range(len(temp_list[i][2])):
                                        test_dict[temp_list[i][2][j]] += temp_list[i][1] #각 index가 얼마나 사용되지 입력
                                        
                                    
                                for i in test_dict.keys(): 
                                    if stock_group_df['횟수'][i] < test_dict[i]: #조합에서 사용되는 양이 사용 가능한 양보다 많을 경우
                                        no_overlap = False
                                        break
                                        
                            if no_overlap:
                                combi_weight = temp_combi_weight
                                best_score = temp_score 
                                select_residual = temp_total_residual
                                #select_addition_weight = temp_addition_weight
                                select_list = temp_list
                        
                    
                    ## total_best 초기화                                          
                    if total_best_score < best_score or len(total_best_select_list)==0:
                        temp_sum = 0
                        for i in range(len(select_list)):
                            temp_sum +=select_list[i][0]
                        
                                        
                        if temp_sum <= cal_realweight :
                            total_best_score = best_score
                            total_best_select_list = select_list
                            total_best_stock_num = n
                            total_best_material_realweight = cal_realweight
                            total_best_residual = select_residual
                            #total_best_addition_weight = select_addition_weight
                            total_best_combi_weight = combi_weight
                
                #### 좋은 재료가 없을 경우
                if total_best_score == -CONST_OUT_OF_COUNT_NUM:
                    #print(sorted_realweight_list)
                    need_new_material_soure +=1
                    combi_try_count +=1
                    
                elif combi_try_count > len(stock_df)*2  : # and total_best_score>-50  and total_best_score>-100 
                    one_more = 0
                    need_new_material_soure = 0
                    ## 재설정
                    select_list = total_best_select_list
                    best_score = total_best_score
                    n = total_best_stock_num
                    select_residual = total_best_residual
                    combi_weight = total_best_combi_weight
                    cal_realweight = total_best_material_realweight
                    weight = float(stock_df['포장중량'][n])/1000
                    width  = stock_df['실폭'][n]
               
                    print("combi_try_count",combi_try_count)
                    combi_try_count = 0 
                    print("standard_score",standard_score)
                    print("best_score",best_score)
                    standard_score = 100
                    print("select_list",select_list)
                    print("select_residual",select_residual)
                        
                    for i in range(len(select_list)):
                        for j in range(len(select_list[i][2])):
                            special_addition_index = False
                            for k in range(len(special_addition_count_list)):
                                if select_list[i][2][j] == special_addition_count_list[k][1]:
                                    special_addition_index = True
                                    break;
                                    
                            if not special_addition_index:         
                                if stock_group_df['횟수'][select_list[i][2][j]] < CONST_OUT_OF_COUNT_NUM:
                                    order_count = (select_list[i][1]+select_list[i][5][j])
                                    order_addition_count = select_list[i][5][j]    
                                    stock_group_df['횟수'][select_list[i][2][j]] -= order_count
                                    big_group_total_count -= order_count
                                    current_count += order_count

                                    stock_group_df['addition_횟수'][select_list[i][2][j]] += order_addition_count
                                    total_addition_count += order_addition_count
                                    if stock_group_df['횟수'][select_list[i][2][j]] <=0:
                                        stock_group_df['횟수'][select_list[i][2][j]] = CONST_OUT_OF_COUNT_NUM    
                                else:
                                    stock_group_df['addition_횟수'][select_list[i][2][j]] -= select_list[i][1]
                                    total_addition_count -= select_list[i][1]
                                    
                    if len(special_addition_count_list) !=0:
                        for i in range(len(special_addition_count_list)):
                            special_addition_count =\
                                special_addition_count_list[i][0] - stock_group_df['횟수'][special_addition_count_list[i][1]]
                            stock_group_df['addition_횟수'][special_addition_count_list[i][1]] -= special_addition_count
                            current_count += stock_group_df['횟수'][special_addition_count_list[i][1]]
                            big_group_total_count -= stock_group_df['횟수'][special_addition_count_list[i][1]]
                            total_addition_count -= special_addition_count
                            stock_group_df['횟수'][special_addition_count_list[i][1]] = CONST_OUT_OF_COUNT_NUM
                        special_addition_count_list = []
                    temp_sum = 0
                    for i in range(group_index_start,group_index_end):
                        if stock_group_df['횟수'][i] != CONST_OUT_OF_COUNT_NUM:
                            temp_sum += stock_group_df['횟수'][i] 
                    #big_group_total_count = temp_sum
                                
                    print("big_group_index range: ",small_group_index[start_index],small_group_index[end_index])
                    print("stock 번호: ",n)
                    #print("실폭: ",const_width)       
                    #print("제품두께: ",thickness)
                    print("width: ",width,"weight:", weight)
                    print("투입량:",cal_realweight)
                    print("조합량:",combi_weight)
                    print("stock_index",n,"실손량: ",select_residual)
                    #제거 목록 index 선택
                    sum_residual += select_residual
                    #sum_addition_weight += select_addition_weight
                    selected_stock.append(n)
                    selected_stock.sort()
                    temp_selected_list = []
                    temp_selected_list.append(n)
                    temp_selected_list.append(select_list)
                    selected_list.append(temp_selected_list)
                        
                    print("stock_index:",n,", 진행률: ",current_count/total_count,",실손량: ",sum_residual,",추가 생산량: ",sum_addition_weight)
                    print("current_count: ",current_count,", total_addition_count: ",total_addition_count)
                    print("big_group_total_count",big_group_total_count)
                    print()
                    n = random.randrange(0,len(stock_df))
                    SUM_OF_SCOURE += total_best_score 
                    SUM_OF_SCOURE_COUNT +=1
                    total_best_score = -CONST_OUT_OF_COUNT_NUM
                    total_best_select_list = []
                    total_best_stock_num = -1
                    total_best_residual = CONST_OUT_OF_COUNT_NUM
                    total_best_combi_weight = 0
                    total_best_material_realweight = 0
                elif combi_try_count > len(stock_df)*2+2 : #total_best_score <-500 and 
                    print("combi_try_count",combi_try_count)
                    combi_try_count =0
                    print("impossible!!!!%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")
                    break;    
                else:  
                    combi_try_count +=1
                    #standard_score += math.log(range_weight*big_group_total_count)*math.log((120-standard_score)/(120-total_best_score))#*100*(combi_try_count/len(stock_df))
print("finish~!!!!!!!!!!!!!!!!!!!!!!!")

small_group_index[start_index],small_group_index[end_index] 0 1
big_group_total_count 22
combi_try_count 111
standard_score 100
best_score 42.47808141726335
select_list [[6.16, 4, [0], [22], [1273.3], [0], 16.700000000000045, 0.08]]
select_residual 1.3399999999999999
big_group_index range:  0 1
stock 번호:  33
width:  1290 weight: 7.978
투입량: 7.58
조합량: 6.16
stock_index 33 실손량:  1.3399999999999999
stock_index: 33 , 진행률:  0.015503875968992248 ,실손량:  1.3399999999999999 ,추가 생산량:  0
current_count:  4 , total_addition_count:  0
big_group_total_count 18



/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:553: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


combi_try_count 111
standard_score 100
best_score 42.47808141726335
select_list [[6.16, 4, [0], [18], [1273.3], [0], 16.700000000000045, 0.08]]
select_residual 1.3399999999999999
big_group_index range:  0 1
stock 번호:  33
width:  1290 weight: 7.978
투입량: 7.58
조합량: 6.16
stock_index 33 실손량:  1.3399999999999999
stock_index: 33 , 진행률:  0.031007751937984496 ,실손량:  2.6799999999999997 ,추가 생산량:  0
current_count:  8 , total_addition_count:  0
big_group_total_count 14

combi_try_count 111
standard_score 100
best_score 42.47808141726335
select_list [[6.16, 4, [0], [14], [1273.3], [0], 16.700000000000045, 0.08]]
select_residual 1.3399999999999999
big_group_index range:  0 1
stock 번호:  33
width:  1290 weight: 7.978
투입량: 7.58
조합량: 6.16
stock_index 33 실손량:  1.3399999999999999
stock_index: 33 , 진행률:  0.046511627906976744 ,실손량:  4.02 ,추가 생산량:  0
current_count:  12 , total_addition_count:  0
big_group_total_count 10

combi_try_count 111
standard_score 100
best_score 42.47808141726335
select_list [[6.16, 4

/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:556: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


combi_try_count 112
standard_score 100
best_score 86.43791793313085
select_list [[6.62, 4, [2, 2], [10, 10], [708.1, 678.1], [0, 0], 3.7999999999999545, 0.01]]
select_residual 0.8899999999999997
big_group_index range:  2 3
stock 번호:  34
width:  1390 weight: 7.918
투입량: 7.52
조합량: 6.62
stock_index 34 실손량:  0.8899999999999997
stock_index: 34 , 진행률:  0.16279069767441862 ,실손량:  8.48 ,추가 생산량:  0
current_count:  42 , total_addition_count:  -2
big_group_total_count 12

combi_try_count 111
standard_score 100
best_score 86.43126899696063
select_list [[6.61, 4, [2, 2], [6, 6], [708.1, 678.1], [0, 0], 3.7999999999999545, 0.02]]
select_residual 0.8899999999999997
big_group_index range:  2 3
stock 번호:  34
width:  1390 weight: 7.918
투입량: 7.52
조합량: 6.61
stock_index 34 실손량:  0.8899999999999997
stock_index: 34 , 진행률:  0.1937984496124031 ,실손량:  9.370000000000001 ,추가 생산량:  0
current_count:  50 , total_addition_count:  -2
big_group_total_count 4

need new material!!!!%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:565: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:569: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


combi_try_count 111
standard_score 100
best_score 10.563706563706559
select_list [[4.36, 2, [10, 10, 10], [18, 18, 18], [572.0, 552.0, 552.0], [0, 0, 0], 34.0, 0.09]]
select_residual -0.009999999999999787
big_group_index range:  9 12
stock 번호:  54
width:  1710 weight: 4.8
투입량: 4.44
조합량: 4.36
stock_index 54 실손량:  -0.009999999999999787
stock_index: 54 , 진행률:  0.7674418604651163 ,실손량:  20.240000000000002 ,추가 생산량:  0
current_count:  198 , total_addition_count:  -24
big_group_total_count 52

combi_try_count 111
standard_score 100
best_score 10.563706563706559
select_list [[4.36, 2, [10, 10, 10], [16, 16, 16], [572.0, 552.0, 552.0], [0, 0, 0], 34.0, 0.09]]
select_residual -0.009999999999999787
big_group_index range:  9 12
stock 번호:  54
width:  1710 weight: 4.8
투입량: 4.44
조합량: 4.36
stock_index 54 실손량:  -0.009999999999999787
stock_index: 54 , 진행률:  0.7906976744186046 ,실손량:  20.230000000000004 ,추가 생산량:  0
current_count:  204 , total_addition_count:  -24
big_group_total_count 46

combi_try_count 

/home/leo/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:558: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


need new material!!!!%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [43]:
stock_group_df

,세부품목,제품코드,폭조합,두께,폭,길이(기준),길이_MIN,길이_MAX,ALLOY_1,ALLOY_2,...,원자재_M,원자재_T,생산량,횟수,const_횟수,addition_횟수,const_생산량,const_addition_횟수,addition_생산량,const_addition_생산량
0,박박,7.2-1238.3X-6,O,7.2,1238.3,31000.0,28000.0,33000.0,AC,0,...,무관,H14,32.0,2,22,0,32.0,12,16.00,16.00
1,박박,9-830AC0KA,O,9.0,830.0,12000.0,12000.0,12000.0,AC,0,...,"X,D",H14,6.0,100000,14,-2,6.0,8,3.00,3.00
2,박박,5.5-673.1AC8X-6,O,5.5,673.1,40000.0,23000.0,42000.0,AC,0,...,"C,Z",H14,16.0,4,20,0,16.0,10,8.00,8.00
3,LIB,50-540-3000AC8-B-6,O,50.0,540.0,3000.0,1800.0,3000.0,AC,0,...,"D,N",H14,1.0,100000,4,-8,1.0,2,0.50,0.50
4,LIB,20-540-7500AC8-B-6,O,20.0,540.0,7500.0,3600.0,7500.0,AC,0,...,"D,N",H14,2.5,100000,6,0,2.5,4,1.25,1.25
5,LIB,20-465-7500AC8-B-6,O,20.0,465.0,7500.0,3600.0,7500.0,AC,0,...,"D,N",H14,2.0,100000,6,-6,2.0,4,1.00,1.00
6,LIB,20-1200AC8SD-B-6P,O,20.0,1200.0,7500.0,3000.0,7500.0,AC,0,...,"D,N",H14,14.0,100000,16,0,14.0,8,7.00,7.00
7,LIB,20-616AC8X-B76,O,20.0,616.0,7500.0,3600.0,7500.0,AC,0,...,"D,N",H14,12.0,2,24,0,12.0,12,6.00,6.00
8,LIB,20-465AC8X-B6,O,20.0,465.0,6000.0,4500.0,6000.0,AC,0,...,"D,N",H14,16.0,100000,54,0,16.0,28,8.00,8.00
9,LIB,12-577AC8SK-B-6P,O,12.0,577.0,20000.0,6000.0,20000.0,AC,0,...,"D,N",H14,8.0,2,12,0,8.0,6,4.00,4.00


In [29]:
order_list = []
for i in range(len(stock_group_df['제품코드'])):
    order_list.append([stock_group_df['제품코드'][i],0,0])
    
## 조합 dataframe, save excel file
combi_df_list = []
for i in range(len(selected_list)):
    meterial_weight = stock_df['포장중량'][selected_list[i][0]]
    temp_list =[]
    temp_list.append(stock_df['index'][selected_list[i][0]])
    temp_list.append(stock_df['제품코드'][selected_list[i][0]])
    temp_list.append(meterial_weight)
    temp_width_sum = []
    temp_weight_list = []
    temp_length_sum = ""
    temp_length = 0
    temp_count_sum = ""
    temp_index_sum = ""
    temp_order_num = []
    temp_width = ""
    for j in range(len(selected_list[i][1])):
        #temp_weight_sum += str(selected_list[i][1][j][0])+"  "
        temp_length_sum += str(stock_group_df['길이(기준)'][selected_list[i][1][j][2][0]])+"  "
        temp_count_sum += str(selected_list[i][1][j][1])+"  "
        temp_index_sum += str(selected_list[i][1][j][2])+"  "
        width_sum =0
        
        for k in range(len((selected_list[i][1][j][2]))):
            temp_order_num.append(stock_group_df['제품코드'][selected_list[i][1][j][2][k]])
            width_sum += float(stock_group_df['폭'][selected_list[i][1][j][2][k]])
            temp_width = stock_group_df['폭'][selected_list[i][1][j][2][k]]
            temp_length = stock_group_df['길이(기준)'][selected_list[i][1][j][2][k]]
            temp_thickness = stock_group_df['두께'][selected_list[i][1][j][2][k]]
            temp_weight_list.append(round(utils.realWeight(temp_thickness,temp_width,temp_length,1,selected_list[i][1][j][1]),2))
            for m in range(len(order_list)):
                if order_list[m][0] == stock_group_df['제품코드'][selected_list[i][1][j][2][k]]:
                    order_list[m][1] += round(utils.realWeight(temp_thickness,temp_width,temp_length,1,selected_list[i][1][j][1]),2)
                    order_list[m][2] = stock_df['제품코드'][selected_list[i][0]]
            
        temp_width_sum.append(width_sum)
        #temp_order_num.append
            #print(float(selected_list[i][1][j][4][k]))
    
    cal_meterial_weight = utils.calculateRealweight(temp_thickness,meterial_weight)
    temp_list.append(cal_meterial_weight)
    weight_list_sum = reduce(lambda x1,x2: x1+x2,temp_weight_list)
    temp_list.append(weight_list_sum)
    temp_list.append(cal_meterial_weight-weight_list_sum)
    temp_list.append(stock_df['실폭'][selected_list[i][0]])
    temp_list.append(temp_width_sum)    
    temp_list.append(stock_df['실폭'][selected_list[i][0]]-temp_width_sum)
    temp_list.append(temp_weight_list)
    temp_list.append(temp_length_sum)
    temp_list.append(temp_count_sum)
    temp_list.append(temp_order_num)
    #temp_list.append(temp_width)
    combi_df_list.append(temp_list)
combi_df = pd.DataFrame(columns =['원자재 index','원자재 제품코드','원자재 포장중량','가공무게','조합 무게합','남은 무게'\
                                  ,'원자재 실폭','주문 실폭합',"폭 차(mim 포함)",'조합 무게','조합 길이','조합 횟수','조합 제품코드'],data=combi_df_list)
combi_df = combi_df.sort_values(['원자재 index'])
combi_df

,원자재 index,원자재 제품코드,원자재 포장중량,가공무게,조합 무게합,남은 무게,원자재 실폭,주문 실폭합,폭 차(mim 포함),조합 무게,조합 길이,조합 횟수,조합 제품코드
18,114,300-1160AC4N,6232,5758.37,2911.19,2847.18,1160,[1119.0],[41.0],"[1501.12, 1410.07]",20000.0,2,"[12-577AC8SK-B-6P, 12-542AC8SK-B-6P]"
9,114,300-1160AC4N,6232,5733.44,5268.24,465.20,1160,[1080.0],[80.0],"[2634.12, 2634.12]",3000.0,6,"[50-540-3000AC8-B-6, 50-540-3000AC8-B-6]"
11,140,300-1230AC4D,7537,6918.97,5853.60,1065.37,1230,[1200.0],[30.0],[5853.6],7500.0,6,[20-1200AC8SD-B-6P]
20,145,300-1230AC4D,7269,6716.56,6004.50,712.06,1230,[1154.0],[76.0],"[3002.25, 3002.25]",20000.0,4,"[12-577AC8SK-B-6P, 12-577AC8SK-B-6P]"
10,145,300-1230AC4D,7269,6672.94,5853.60,819.34,1230,[1200.0],[30.0],[5853.6],7500.0,6,[20-1200AC8SD-B-6P]
0,194,300-1290AC4N,7978,7579.10,5992.10,1587.00,1290,[1238.3],[51.700000000000045],[5992.1],31000.0,4,[7.2-1238.3X-6]
1,194,300-1290AC4N,7978,7579.10,5992.10,1587.00,1290,[1238.3],[51.700000000000045],[5992.1],31000.0,4,[7.2-1238.3X-6]
2,194,300-1290AC4N,7978,7579.10,5992.10,1587.00,1290,[1238.3],[51.700000000000045],[5992.1],31000.0,4,[7.2-1238.3X-6]
3,194,300-1290AC4N,7978,7579.10,5992.10,1587.00,1290,[1238.3],[51.700000000000045],[5992.1],31000.0,4,[7.2-1238.3X-6]
4,194,300-1290AC4N,7978,7579.10,5992.10,1587.00,1290,[1238.3],[51.700000000000045],[5992.1],31000.0,4,[7.2-1238.3X-6]


In [30]:
code_producted_weight = []
for i in range(len(order_list)):
    code_producted_weight.append(order_list[i][0])

code_producted_weight = list(set(code_producted_weight))
code_producted_weight.sort()
code_producted_weight_list =[]
for i in range(len(code_producted_weight)):
    temp_list = [code_producted_weight[i],0,0,0,[],'','','']
    for j in range(len(order_list)):
        if temp_list[0] == order_list[j][0]:
            temp_list[1] += order_list[j][1]
            temp_list[4].append(order_list[j][2])
    temp_list[1] = round(temp_list[1],3)
    for j in range(len(stock_group_df['제품코드'])):
        if temp_list[0] == order_list[j][0]:
            temp_list[2] = round(stock_group_df['생산량'][j]*1000,3)
    temp_list[3] = round(temp_list[2]-temp_list[1],3)
    temp_list[4] = list(set(temp_list[4]))

    code_producted_weight_list.append(temp_list)
extra_sum =0
total_sum =0.0
need_sum =0
result_df =  pd.DataFrame(columns =['제품코드','조합량','주문량','필요생산량(<0:추가생산량)','투입 원자재','','',''],\
                          data=code_producted_weight_list)
for i in range(len(result_df)):
    if result_df['필요생산량(<0:추가생산량)'][i]<0:
        extra_sum+= result_df['필요생산량(<0:추가생산량)'][i]
    else:
        need_sum +=result_df['필요생산량(<0:추가생산량)'][i] 
    total_sum += (result_df['주문량'][i])
    
residual_sum = 0
for i in range(len(combi_df)):
    residual_sum += float(combi_df["남은 무게"][i])
    
print("총 낭비량",residual_sum)
print("총 추가 생산량",extra_sum,"미 생산량",need_sum,"주문량",total_sum)
temp_list =['','','','','','','','']
code_producted_weight_list.append(temp_list)
temp_list =['','','','','','','','']
code_producted_weight_list.append(temp_list)

temp_list =["총 추가 생산량",-int(extra_sum),"미 생산량",int(need_sum),"주문량",total_sum,"총 낭비량",int(residual_sum)]

code_producted_weight_list.append(temp_list)
result_df =  pd.DataFrame(columns =['제품코드','조합량','주문량','필요생산량(<0:추가생산량)','투입 원자재','','',''],\
                          data=code_producted_weight_list)
result_df


총 낭비량 28233.189999999988
총 추가 생산량 -13912.160000000002 미 생산량 7516.76 주문량 159500.0


,제품코드,조합량,주문량,필요생산량(<0:추가생산량),투입 원자재,,,
0,12-352AC8EU-B76,9157.65,10000,842.35,[230-1850AC4N],,,
1,12-542AC8SK-B-6P,43712.2,40000,-3712.17,[300-1710AC4D],,,
2,12-577AC8SK-B-6P,7505.62,8000,494.38,[300-1230AC4D],,,
3,20-1200AC8SD-B-6P,15609.6,14000,-1609.6,[300-1290AC4N],,,
4,20-465-7500AC8-B-6,2268.27,2000,-268.27,[300-1460AC4N],,,
5,20-465AC8X-B6,18146.2,16000,-2146.16,[300-1460AC4N],,,
6,20-540-7500AC8-B-6,2634.12,2500,-134.12,[300-1710AC4D],,,
7,20-616AC8X-B76,11017.8,12000,982.21,[300-1290AC4N],,,
8,5.5-673.1AC8X-6,12841.7,16000,3158.32,[300-1390AC4Z],,,
9,50-540-3000AC8-B-6,5268.24,1000,-4268.24,[300-1160AC4N],,,


In [40]:
result_score = 100-100*((need_sum/total_sum) + (extra_sum/(total_sum-need_sum)) + (residual_sum/(total_sum-need_sum)))
result_score

85.86452855819141

-2032.32 28944.920000000002 67000.0


In [25]:
now = datetime.now()
time = str(now.year)+'_'+str(now.month)+'_'+str(now.day)+'_'+str(now.hour)+'_'+str(now.minute)+'_'
file_name=time+group_name+"_result.xlsx"
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
combi_df.to_excel(writer,sheet_name ='combi_result')
result_df.to_excel(writer,sheet_name ='product_result')
writer.close()